In [1]:
import torch
import tifffile
import glob
import numpy as np
from tqdm import tqdm  
import os
from Models.LargePNet import LargePNet

C:\Users\Dell\.conda\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Directory of the testing data
head_dir = r"Data\STED_denoising\MT\Testing"
Raw_path = head_dir + r'\TestNoisy'
all_imgs_path = glob.glob(Raw_path + '\\'+'*.tif')
num = len(all_imgs_path)
Endstr ='.tif'
device = 'cuda'
save_path = head_dir + '\\'+'output'
if not os.path.exists(save_path):  
    os.mkdir(save_path)

In [5]:
model_label = r'LargePNet'
model =  LargePNet(1, 1, 2, 15, 4)
# Users need to specify the model dir
model.load_state_dict(torch.load(r'Data\STED_denoising\MT\logfile\LargePNet\best_model.pth'))
model_label = model_label
save_path = head_dir + '\\'+'output'+'\\'+model_label
if not os.path.exists(save_path):  
    os.mkdir(save_path)

pbar = tqdm(total=num, desc="Processing images")
for i in range(num):     
    numstr = np.array2string(np.array(i+1))
    I = tifffile.imread(Raw_path+ '\\' +numstr+Endstr)
    I = np.array(I)
    #I = I/I.max()
    I = (I - I.min()) / (I.max() - I.min())
    im = torch.tensor(I, dtype=torch.float32)
    im = im.unsqueeze(0).unsqueeze(0)
    imnose = np.transpose(im.data.numpy(),(0,2,3,1))
    imnose = imnose[0,...]
    model = model.to(device)
    with torch.no_grad():
        model.eval()
        #with autocast():
        output = model(im.to(device))
        output = output.to('cpu')
        output = output.detach()
        imde = np.transpose(output.data.numpy(),(0,2,3,1))
        imdeHybrid = imde[0,...]  
        imde = imdeHybrid
        imde = (2**16-1)*imde
        #imde = (2**16-1)*(imde-imde.min())/(imde.max()-imde.min())
        imde = imde.astype(np.uint16)
        tifffile.imwrite(save_path+ '\\' +numstr+model_label+Endstr,imde)
    pbar.update(1)
pbar.close()

drop path: Identity()
drop path: Identity()
drop path: Identity()


Processing images: 100%|██████████| 135/135 [00:27<00:00,  4.88it/s]
